In [1]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split

This is an example from https://www.freecodecamp.org/news/how-to-extract-keywords-from-text-with-tf-idf-and-pythons-scikit-learn-b2a0f3d7e667/

In [2]:
df_idf = pd.read_csv('amazon/reviews.csv')

In [3]:
print("Schema:\n", df_idf.dtypes)
print("Shape of database =", df_idf.shape)

Schema:
 asin             object
name             object
rating            int64
date             object
verified           bool
title            object
body             object
helpfulVotes    float64
dtype: object
Shape of database = (82815, 8)


In [4]:
def pre_process(text):
    # to lowercase
    text=text.lower()
    
    # remove tags
    text = re.sub("&lt;/?.*?&gt;", "&lt;&gt; ", text)
    
    # remove special characters and digits
    text = re.sub("(\\d|\\W)+", " ", text)
    
    return text

In [5]:
df_idf['text'] = df_idf['title'] + " " + df_idf['body']
df_idf['text'] = df_idf['text'].apply(lambda x: pre_process(str(x)))

In [6]:
df_idf['text'][2]

'love this phone this is a great reliable phone i also purchased this phone after my samsung a died the menu is easily comprehendable and speed dialing is available for around numbers voice dialing is also a nice feature but it takes longer than speed dialing the only thing that bothers me is the games nokia seems to have taken snake and off their phones there is a skydiving game bowling and tennis like pong the ringers are very nice and a feature is available to choose a different ringer for each person calling however ringtones are not available online to download to this phone you re pretty much stuck with what you have there are vibrating ringtones and regular midi polyphonic tones all they need are covers in a reasonable price range '

In [7]:
def get_stop_words(stop_file_path):
    with open(stop_file_path, 'r', encoding='utf-8') as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return frozenset(stop_set)

In [8]:
stopwords = get_stop_words('stopwords.txt')
docs = df_idf['text'].tolist()

In [9]:
cv = CountVectorizer(max_df = .85, stop_words=stopwords)
wordCountVec = cv.fit_transform(docs)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['come', 'vis', 'viser', 'visest'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [10]:
list(cv.vocabulary_.keys())[:10]

['def',
 'best',
 'worst',
 'samsung',
 'awhile',
 'absolute',
 'doo',
 'read',
 'review',
 'detect']

In [11]:
tfidf = TfidfTransformer(smooth_idf=True, use_idf=True)
tfidf.fit(wordCountVec)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [12]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1] , x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    sorted_items = sorted_items[:topn]
    
    score_vals = []
    feature_vals = []
    
    for idx, score in sorted_items:
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
        
        results = {}
        for idx in range(len(feature_vals)):
            results[feature_vals[idx]]=score_vals[idx]
            
        return results

In [13]:
feature_names = cv.get_feature_names()

doc = docs[1]

tf_idf_vector = tfidf.transform(cv.transform([doc]))

sorted_items = sort_coo(tf_idf_vector.tocoo())

keywords = extract_topn_from_vector(feature_names, sorted_items, 10)


In [42]:
test = cv.transform([doc])

print(keywords)
# for idx in range(len(sorted_items)):
#     print(feature_names[sorted_items[idx][0]], sorted_items[idx][1])

{'sprint': 0.442}


In [15]:
y = df_idf['rating']
# fixing the labels, if > 3.5 is going to be 1 which is positive, else 0
y = y.apply(lambda x: 1 if x > 3.5 else 0) 
y = y.to_numpy()
x = wordCountVec.toarray()
print(x.shape, y.shape)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.5)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(82815, 34848) (82815,)
(41407, 34848) (41407,)
(41408, 34848) (41408,)


In [16]:
print(X_train[:, :], y_train[:])

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] [1 1 1 ... 1 0 1]


In [17]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [18]:
# Save the model in a binary file
import pickle
filename = 'model.sav'
pickle.dump(clf, open(filename, 'wb'))

In [19]:
# Loads the model from the binary file
loaded_model = pickle.load(open(filename, 'rb'))

In [21]:
print(X_test[:, 0], y_test[0])

[0 0 0 ... 0 0 0] 0


In [28]:
y_train_p = clf.predict(X_train)
y_test_p = clf.predict(X_test)

In [32]:
# for i in range(len(y_test)):
#     print(y_test_p[i], y_test[i])

In [65]:
from sklearn.metrics import accuracy_score, confusion_matrix
print("Accuracy in testing set:", accuracy_score(y_test, y_test_p))
print("Accuracy in training set:", accuracy_score(y_train, y_train_p))
print(confusion_matrix(y_test, y_test_p))
print(confusion_matrix(y_train, y_train_p))

Accuracy in testing set: 0.8741547527047914
Accuracy in training set: 0.9873209843746227
[[10945  2701]
 [ 2510 25252]]
[[13155   292]
 [  233 27727]]


In [64]:
test = cv.transform(["This is the best device", "I hate this phone"]).toarray()
clf.predict(test)

array([1, 0])